In [46]:
import datetime
import numpy as np
import open3d as o3d
import networkx as nx
from Graph_Path import array_to_graph, extract_path_info
from LS_circle import getRootPt
from ExtractInitWood import extract_init_wood
from ExtractFinalWood import extract_final_wood
from Accuracy_evaluation import evaluate_indicators
from Visualization import show_graph, sp_graph, show_pcd

In [47]:
# load single tree point cloud.
pcd = o3d.io.read_point_cloud('C:\\leafonforgbs\\TD203M.pcd')
pcd = np.asarray(pcd.points)

In [48]:
# Please ensure that the growth direction of the tree is parallel to the Z coordinate axis.
treeHeight = np.max(pcd[:, 2])-np.min(pcd[:, 2])

In [49]:
# fit the root point.
root, fit_seg = getRootPt(pcd, lower_h=0.0, upper_h=0.2)
pcd = np.append(pcd, root, axis=0)
root_id = pcd.shape[0]-1
print("root_ID:", root_id)
# show_pcd(pcd)

root_ID: 3000000


In [50]:
# construct networkx Graph.
print(str(datetime.datetime.now()) + ' | >>>constructing networkx Graph...')
G = array_to_graph(pcd, root_id, kpairs=3, knn=300, nbrs_threshold=treeHeight/30, nbrs_threshold_step=treeHeight/60)

2024-10-22 17:11:37.272909 | >>>constructing networkx Graph...
unprocessed_count: 2999627
unprocessed_count: 2999545
unprocessed_count: 2999447
unprocessed_count: 2999346
unprocessed_count: 2999269
unprocessed_count: 2999196
unprocessed_count: 2999133
unprocessed_count: 2999069
unprocessed_count: 2999021
unprocessed_count: 2998975
unprocessed_count: 2998925
unprocessed_count: 2998871
unprocessed_count: 2998817
unprocessed_count: 2998771
unprocessed_count: 2998741
unprocessed_count: 2998713
unprocessed_count: 2998684
unprocessed_count: 2998662
unprocessed_count: 2998640
unprocessed_count: 2998615
unprocessed_count: 2998582
unprocessed_count: 2998543
unprocessed_count: 2998501
unprocessed_count: 2998458
unprocessed_count: 2998413
unprocessed_count: 2998371
unprocessed_count: 2998321
unprocessed_count: 2998264
unprocessed_count: 2998203
unprocessed_count: 2998140
unprocessed_count: 2998075
unprocessed_count: 2998019
unprocessed_count: 2997960
unprocessed_count: 2997889
unprocessed_count: 

In [51]:
# # save/read already constructed Graph to reduce processing time.
# Save the graph
with open('C:\\leafsepAU\\ql12.gpickle', 'wb') as f:
    pickle.dump(G, f)

# Load the graph
with open('C:\\leafsepAU\\ql12.gpickle', 'rb') as f:
    G = pickle.load(f)

In [52]:
import pickle

In [53]:
print(">>>connected components of constructed Graph: ", nx.number_connected_components(G))
show_graph(pcd, G)

>>>connected components of constructed Graph:  1


In [54]:
# extract path info information from graph
print(str(datetime.datetime.now()) + ' | >>>extracting shortest path information...')
path_dis, path_list = extract_path_info(G, root_id, return_path=True)
show_graph(pcd, sp_graph(path_list, root_id))

2024-10-22 17:17:54.302114 | >>>extracting shortest path information...


In [55]:
# extract initial wood points.
print(str(datetime.datetime.now()) + ' | >>>extracting initial wood points...')
init_wood_ids = extract_init_wood(pcd, G, root_id, path_dis, path_list,
                                  split_interval=[0.1, 0.2, 0.3, 0.5, 1], max_angle=0.25*np.pi)

2024-10-22 17:19:57.654624 | >>>extracting initial wood points...
cut edges...
interval: 0.1
components: 691974
interval: 0.2
components: 650543
interval: 0.3
components: 636969
interval: 0.5
components: 626823
interval: 1
components: 618164


In [56]:
# extract final wood points.
print(str(datetime.datetime.now()) + ' | >>>extracting final wood points...')
final_wood_mask = extract_final_wood(pcd, root_id, path_dis, path_list, init_wood_ids, G)

2024-10-22 17:23:22.521502 | >>>extracting final wood points...


In [57]:
# remove the inserted root point and extract wood/leaf points by mask index.
final_wood_mask[-1] = False
wood = pcd[final_wood_mask]
final_wood_mask[-1] = True
leaf = pcd[~final_wood_mask]

In [58]:
# write separation result with .txt format.
np.savetxt('C:\\leafonforgbs\\QL_12wood_points.txt', wood, fmt='%1.6f')
np.savetxt('C:\\leafonforgbs\\QL_12leaf_points.txt', leaf, fmt='%1.6f')   